In [1]:
import numpy as np
from numba import jit
import timeit

In [2]:
def nk( maxki ):
    """Combinatorics, 
    returns number of combis for lattice vectors in 3D,
    (0,0,0), (0,0,1), (0,0,-1), (0,1,0), ..., (0,1,1), (0,1,-1), (0,-1,1), (1,0,1), ..., (1,1,1), ...
    """
    mitnull = 4 * (maxki + 1) ** 3 
    #korr = 7 + sum([30 + i * 24 for i in range(maxki)])
    korr = 3 + 6 * maxki + 3 * maxki ** 2
    return mitnull - korr

In [3]:
assert [nk(i) for i in range(3)] == [1, 20, 81]

In [4]:
@jit
def kvec_jit( maxki=3 ):#, maxnk=300, maxr=0):
    """Returns k-vectors for fourier part of ewald 
    
    Arguments:
        maxki (int): max value for kx, ky, kz
        maxnk (int): max # of k vectors, unrestrict with maxnk=0 
        maxr  (int): max length of k, unrestrict with maxr=0
    
    Output:
        k (float): k vectors (dim = 3 x maxnk)
    """
    
    vec = np.zeros([nk(maxki), 3], dtype=np.float32)
    cnt = 0
    #maxr = maxr**2
    
    vec[cnt] = (0,0,0)
    cnt += 1
    for i in range(1, maxki + 1):
        vec[cnt], vec[cnt+1] = [0,0,i], [0,0,-i]
        vec[cnt+2], vec[cnt+3] = [0,i,0], [0,-i,0]
        vec[cnt+4], vec[cnt+5] = [i,0,0], [-i,0,0]
        cnt += 6
    for i in range(1, maxki + 1):
        for j in range(1, maxki + 1):
            vec[cnt], vec[cnt+1], vec[cnt+2] = [0,i,j], [0,-i,j], [0,i,-j]
            vec[cnt+3], vec[cnt+4], vec[cnt+5] = [i,0,j], [-i,0,j], [i,0,-j]
            vec[cnt+6], vec[cnt+7], vec[cnt+8] = [i,j,0], [-i,j,0], [i,-j,0]
            cnt += 9
    for i in range(1, maxki + 1):
        for j in range(1, maxki + 1):
            for k in range(1, maxki + 1):
                    vec[cnt] = [i, j, k]
                    vec[cnt+1] = [-i,j,k]
                    vec[cnt+2] = [i,-j,k]
                    vec[cnt+3] = [i,j,-k]
                    cnt += 4
    return vec

In [5]:
def kvec( maxki=3 ):#, maxnk=300, maxr=0):
    """Returns k-vectors for fourier part of ewald 
    
    Arguments:
        maxki (int): max value for kx, ky, kz
        maxnk (int): max # of k vectors, unrestrict with maxnk=0 
        maxr  (int): max length of k, unrestrict with maxr=0
    
    Output:
        k (float): k vectors (dim = 3 x maxnk)
    """
    
    vec = np.zeros([nk(maxki), 3], dtype=np.float32)
    cnt = 0
    #maxr = maxr**2
    
    vec[cnt] = (0,0,0)
    cnt += 1
    for i in range(1, maxki + 1):
        vec[cnt], vec[cnt+1] = [0,0,i], [0,0,-i]
        vec[cnt+2], vec[cnt+3] = [0,i,0], [0,-i,0]
        vec[cnt+4], vec[cnt+5] = [i,0,0], [-i,0,0]
        cnt += 6
    for i in range(1, maxki + 1):
        for j in range(1, maxki + 1):
            vec[cnt], vec[cnt+1], vec[cnt+2] = [0,i,j], [0,-i,j], [0,i,-j]
            vec[cnt+3], vec[cnt+4], vec[cnt+5] = [i,0,j], [-i,0,j], [i,0,-j]
            vec[cnt+6], vec[cnt+7], vec[cnt+8] = [i,j,0], [-i,j,0], [i,-j,0]
            cnt += 9
    for i in range(1, maxki + 1):
        for j in range(1, maxki + 1):
            for k in range(1, maxki + 1):
                    vec[cnt] = [i, j, k]
                    vec[cnt+1] = [-i,j,k]
                    vec[cnt+2] = [i,-j,k]
                    vec[cnt+3] = [i,j,-k]
                    cnt += 4
    return vec

In [6]:
maxki = 15

%timeit kvec_jit(maxki)
%timeit kvec(maxki)

1.36 ms ± 43.5 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
15.2 ms ± 45.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
def kvec_if( maxki=3 ):#, maxnk=300, maxr=0):
    """Returns k-vectors for fourier part of ewald 
    
    Arguments:
        maxki (int): max value for kx, ky, kz
        maxnk (int): max # of k vectors, unrestrict with maxnk=0 
        maxr  (int): max length of k, unrestrict with maxr=0
    
    Output:
        k (float): k vectors (dim = 3 x maxnk)
    """
    
    vec = np.zeros([nk(maxki), 3], dtype=np.float32)
    cnt = 0
    #maxr = maxr**2
    
    for i in range(maxki + 1):
        for j in range(maxki + 1):
            for k in range(maxki + 1):
                    vec[cnt] = [i, j, k]
                    temp = 0
                    for l in range(3):
                        if vec[cnt][l] > 0:
                            vec[cnt+1+temp] = vec[cnt]
                            vec[cnt+1+temp][l] = -vec[cnt][l] 
                            temp += 1
                    cnt += 1 + temp
    return vec

In [8]:
maxki = 15

%timeit kvec_if(maxki)

40.8 ms ± 315 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
#import itertools as it
#comb = list(it.product(range(1,3), repeat=3))
#print(comb)